<h1 style="color:brown;">  Recurrent neural nets</h1> 

![Looping network](./img/RNN_colah.png)

##### RNNs can produce amazing results <a href ="http://karpathy.github.io/2015/05/21/rnn-effectiveness/">blog</a>

### Lesson plan 
1. Why classic neural nets are not enough?
2. Word embeddings - word2vec
3. Categorical embeddings
4. RNN 
5. Takeaways
6. Hands on word2vec

In [1]:
import numpy as np

### Classic nets vs. RNN's

Classic:
    - Inputs and outputs must be fixed-sized vectors
    - No idea of location or time 

RNNs: 

![](./img/diags.jpeg)

### Word embeddings

#### N-gram

-Which word produces the highest probability to be next given we have seen n specific other words before

In [ ]:
Words: Thank, you, Hello, goodbye

In [ ]:
If we have 4 words and we are looking at 2-gram? 
    Example: no. of times Thank you occurs divided by number of times Thank occurs

We need to calculate the probabilty of 
 - Thank Hello
 - Thank you
 - Thank goodbye
 - Thank Thank

So we needed to do 4 calculations

In [2]:
def how_many_calc_to_do(gram, voc_size):
    '''This function needs to calculate all combos 
    of words'''
    
    return np.prod(np.repeat(voc_size, gram))

In [3]:
how_many_calc_to_do(7, 10000)
# Notice that this is only an approxiamtion and it can be implemented in more efficient ways.

4477988020393345024

![](./img/one_hot_encoding_distance_on_3d.png)

#### Insight I: 
    we can actually just turn each word to a random vector sized 100/200/300, 
    train a classic neural net to predict the next word and update both the weights and the random vectors.
    You can think of it as just another layer of weights multiplying the one hot encoded inputs.
<a href="http://hunterheidenreich.com/blog/intro-to-word-embeddings/">word_embed blog</a>

<a href="https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa">word_embed blog II</a>


![W2V](./img/w2v.png)

##### Objective: maximize the sum of probabilities of each word given its observed window

This idea is very strong in comparison to other options: 

    Bag of words - just count occourences 
    TF-IDF - word is either informative or not but has no relation to other words
    one-hot encoder: for the computer paris-france is the same distance as paris-blabla

- Distance and direction are meaningful! King - man = Queen - woman
- Now the words massive and huge are similar!
- Extends to sentences, paragraphs and documents.

![](./img/see_attached_word_embed.png)

##### We have reduced the dimension of the vocublary by a big factor!
example: from 80,000 to 300

![](https://www.lemay.ai/demo/wordEmbedding/)

##### Insight I.I  The same thing can be applied to any categorical variable. 
##### With enough training data we can learn its continous position in space - state of the art

![categorical_embed](./img/categorical_embedding.png) # image I

![german_states](./img/german_states_mapped_2D.png) 

#### Insight II: 
        well, even if we can include many words (large n-gram), how can we capture context?
        If the text mentioned queen Mary and few pages later is talking about the queen, how will our network 
        know her name is Mary? 

### Idea I: Memory - your current choices are based on previous understanding

Add some cell in the network to keep previous memory and combine with current input to predict next word

![](./img/memory_rnn.png)

#### Problem: calculating the derivative (aka gradient) is problematic, either infinite or zero.

Imagine the memory at time t is the memory at time t-1 times a weight vector:
    $h_t = W*h_{t-1}$
Then:
    $h_t = W^t * h_0$ 
    
  $W > 1$ $h_t --> \infty$

### Solution: LSTM/GRU

<a href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/">LSTM/GRU blog</a>


![](./img/RNNs.png)

![](./img/LSTM_colah.png)

#### Idea II: gates: don't multiply, use addition for memory!

##### Components

    - cell state
    - candidates  

##### Gates
- forget - information to throw (0 means throw all from the cell state)
- input - what values we are going to update
- output - filter which values of the cell we are going to output 

The current cell state is the sum of forgetting and updating with new candidates

### Extension: attention

<a href="https://www.youtube.com/watch?v=SysgYptB198">Intuition</a>

######  - Translate part by part
###### -  Use attention weights - how much attention should you give to each word in the input (update weights to each new word)

![](./img/attention.png)

### Takeaways:
    

##### Word embeddings
- Word/categorical embeddings gives meaning to words in relation to one another
- Word/categorical embeddings are computationally efficient
- Training is done through a classic NN with small window around words

##### RNN
- Old generation RNNs suffered from exploding/vanishing gradients
- New generation RNNs (commonly LSTM or GRU) are using memory gates to mitigate this problem
- RNNs are just multiple copies of a NN connected by the hidden layer
- Training is done again by backpropogation
- Weights are shared accros all network
- RNN's can be used for any sequence. Unlike time series models can include both time and features.
- Are flexible in input and output sizes
- Amazing results in NLP, recommendations and many more.
- Many flavours - BRNN, CRNN...

##### Attention
- Typicall for translations/images
- Weight all the words in one language to decide how much they should influence input to translated language
- components: word weights, BRNN, RNN, context vectors.

### Hands on Word2Vec/word embedding

In [4]:
!pip install gensim

  Stored in directory: /Users/jon/Library/Caches/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built bz2file


In [5]:
import gensim
import numpy as np
import json
import string

##### Reading in the data

In [6]:
with open('JEOPARDY_QUESTIONS1.json') as f:
    data = json.load(f)

In [7]:
len(data)

216930

In [8]:
# Let's look at the first element in our list
data[0]

{'category': 'HISTORY',
 'air_date': '2004-12-31',
 'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
 'value': '$200',
 'answer': 'Copernicus',
 'round': 'Jeopardy!',
 'show_number': '4680'}

In [9]:
# Word2Vec requires that our text have the form of a list
# of 'sentences', where each sentence is itself a list of
# words. How can we put our _Jeopardy!_ clues in that shape?

text = []

for clue in data:
    sentence = clue['question'].translate(str.maketrans('', '',
                                                        string.punctuation)).split(' ')
    
    new_sent = []
    for word in sentence:
        new_sent.append(word.lower())
    
    text.append(new_sent)

In [10]:
# Let's check the new structure of our first clue
text[0]

['for',
 'the',
 'last',
 '8',
 'years',
 'of',
 'his',
 'life',
 'galileo',
 'was',
 'under',
 'house',
 'arrest',
 'for',
 'espousing',
 'this',
 'mans',
 'theory']

#####  Constructing the model

In [11]:
# simply a matter of
# instantiating a Word2Vec object.
model = gensim.models.Word2Vec(text, sg=1)
## sg means skip-gram

##### training 

In [12]:
# To train, call 'train()'!
model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(11337324, 15849970)

In [13]:
# The '.wv' attribute stores the word vectors
model.wv

In [14]:
model.wv['child']

array([-1.7743009e-01,  1.4484370e-01,  9.3734212e-02, -4.1861472e-01,
       -6.7781366e-02,  2.9096228e-01,  2.0395963e-01,  4.4627026e-01,
        3.5138341e-04,  5.0257817e-03, -8.8590890e-02,  2.3621009e-01,
        4.9175479e-02,  3.7999126e-01,  9.2875212e-02, -1.5810581e-01,
        5.6764424e-01, -3.2516658e-01,  4.4945118e-01, -3.9803747e-02,
       -1.6991140e-02,  6.0898262e-01,  3.5365680e-01, -9.2984706e-01,
       -2.2215913e-01,  4.9310226e-02,  4.8305467e-02,  2.2495875e-01,
       -1.3135965e-01, -3.5769451e-01,  4.8653334e-01,  5.4739166e-02,
        1.7923588e-01, -1.7729509e-01,  2.9537898e-01, -5.0075793e-01,
       -2.4204971e-02, -6.4402051e-02, -3.1163868e-01,  1.4211130e-01,
        1.1928944e-01,  3.3671731e-01, -2.1281394e-01,  2.1134712e-01,
       -8.4556752e-01, -1.6900565e-01, -1.6961937e-01, -1.8624520e-01,
        3.5867372e-01,  3.5848536e-02,  4.2364565e-01, -5.8288956e-01,
        2.0673324e-01,  3.5535574e-01,  4.8856390e-01, -9.6862167e-03,
      

In [ ]:
### model.wv methods
#### 'most_similar()' and 'similarity()'

In [15]:
model.wv.most_similar('happiness')

[('prosperity', 0.7474037408828735),
 ('ignorance', 0.744020938873291),
 ('autumns', 0.7291383743286133),
 ('anguish', 0.7205519676208496),
 ('compassion', 0.719251275062561),
 ('arise', 0.7164921760559082),
 ('shakespearebr', 0.7132083177566528),
 ('wherefore', 0.7123302221298218),
 ('goest', 0.7049089670181274),
 ('kindness', 0.7041229605674744)]

In [16]:
model.wv.most_similar('furniture')

[('artwork', 0.7191377878189087),
 ('pottery', 0.7097587585449219),
 ('ceramic', 0.7092421650886536),
 ('decorative', 0.70761638879776),
 ('chippendale', 0.6891075372695923),
 ('monogram', 0.6869503259658813),
 ('wearers', 0.6847445964813232),
 ('flooring', 0.6815633177757263),
 ('fasteners', 0.6767991185188293),
 ('wicker', 0.6764888763427734)]

In [17]:
model.wv.similarity('furniture', 'jewelry')

0.6238540201781528

In [21]:
model.wv.most_similar(positive=['cat', 'animal', 'pet', 'mammal'])

[('reptile', 0.8152649402618408),
 ('shorthaired', 0.7939906120300293),
 ('carnivore', 0.7861495018005371),
 ('pachyderm', 0.7830628156661987),
 ('rodent', 0.7812258005142212),
 ('marsupial', 0.7785536646842957),
 ('parrot', 0.7755876779556274),
 ('giraffe', 0.7702314853668213),
 ('weasel', 0.7701903581619263),
 ('flightless', 0.769274115562439)]

In [19]:
model.wv.most_similar(positive=['cat', 'animal'], negative='pet')

[('breed', 0.39183005690574646),
 ('insect', 0.3801022469997406),
 ('lizard', 0.3366336226463318),
 ('rodents', 0.3326495885848999),
 ('puppy', 0.3324885666370392),
 ('cows', 0.3302050828933716),
 ('dog', 0.32511675357818604),
 ('extinction', 0.3231205642223358),
 ('dogs', 0.32211077213287354),
 ('species', 0.318921834230423)]

In [22]:
model.wv.most_similar(positive=['king', 'woman'], negative='man', topn=3)

[('throne', 0.25100433826446533),
 ('pharaoh', 0.24798095226287842),
 ('empress', 0.24761396646499634)]

In [23]:
model.wv.most_similar(positive='usa')

[('pageant', 0.6598982810974121),
 ('fargo', 0.6350732445716858),
 ('tyra', 0.6186401844024658),
 ('xmas', 0.6108678579330444),
 ('npr', 0.6105656623840332),
 ('dogpatch', 0.6071796417236328),
 ('goings', 0.6024764776229858),
 ('guides', 0.6013813614845276),
 ('2000s', 0.5987871885299683),
 ('surfin', 0.5970957279205322)]

In [24]:
model.wv.most_similar('canada')

[('commonwealth', 0.7130832672119141),
 ('marianas', 0.6783615350723267),
 ('turkmenistan', 0.6684412956237793),
 ('timor', 0.6626874208450317),
 ('territories', 0.6608656644821167),
 ('britain', 0.6576970815658569),
 ('everglades', 0.6575170755386353),
 ('albania', 0.657362699508667),
 ('uruguay', 0.657297670841217),
 ('zambia', 0.6505134701728821)]

In [25]:
model.wv.most_similar('shakespeare')

[('sophocles', 0.7417839169502258),
 ('euripides', 0.7408822774887085),
 ('shakespeares', 0.7374973893165588),
 ('falstaff', 0.6937141418457031),
 ('moliere', 0.690405547618866),
 ('shakespearean', 0.6891647577285767),
 ('tragedy', 0.6753336191177368),
 ('rur', 0.6685926914215088),
 ('karel', 0.6617562770843506),
 ('ibsen', 0.6611332297325134)]

In [33]:
model.wv.most_similar(positive=['president', 'germany'], negative='usa')

[('officially', 0.20530612766742706),
 ('dictator', 0.19773559272289276),
 ('yugoslavia', 0.195656880736351),
 ('emperors', 0.19515357911586761),
 ('czar', 0.1923864781856537),
 ('regime', 0.1913895159959793),
 ('shah', 0.183910071849823),
 ('khmer', 0.18301036953926086),
 ('empires', 0.1802889108657837),
 ('assassinated', 0.180212140083313)]

In [23]:
#### 'doesnt_match()'

In [30]:
model.wv.doesnt_match(['breakfast', 'lunch', 'frog', 'food'])

/anaconda3/envs/learn-env/lib/python3.6/site-packages/gensim/models/keyedvectors.py:730: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'frog'